In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import os
import warnings
import functools
import seaborn as sns
import scipy.stats
import anndata
import dandelion as ddl
import tables

In [2]:
from collections import defaultdict

In [3]:
#Create blank dictionaries for data import.
adata_tmp = defaultdict(dict)
adata_tmp2 = defaultdict(dict)

In [4]:
os.chdir("/Users/matthewcoates/Documents/Cambridge/R files/")


In [5]:
#Import metadata
sampleList = pd.read_csv('/Users/matthewcoates/Documents/Cambridge/COVID_NASAL_test_meta_cellbender.csv')
sampleList

,study_id/patient_id,sampleid,site,cohort,Sub-cohort,batch,Nasal inflammation,COVID Severity,age,cohort.1,Gender,sangerid_GEX,sangerid_BCR,sangerid_TCR
0,N01-001,Sample_Fq21,Nasal,Control,GPA control,2,yes,NaN,middle,COMBIVAS,NaN,NaN,NaN,NaN
1,N01-002,Sample_Fq2,Nasal,Control,GPA control,1,yes,NaN,middle,COMBIVAS,NaN,NaN,NaN,NaN
2,N01-005,Sample_Fq27,Nasal,Control,GPA control,2,yes,NaN,old,COMBIVAS,NaN,NaN,NaN,NaN
3,N01-011,Sample_Fq61,Nasal,Control,GPA control,4,yes,NaN,middle,COMBIVAS,NaN,NaN,NaN,NaN
4,N01-001,Sample_Fq20,Blood,Control,GPA control,2,yes,NaN,middle,COMBIVAS,NaN,NaN,NaN,NaN
5,N01-002,Sample_Fq1,Blood,Control,GPA control,1,yes,NaN,middle,COMBIVAS,NaN,NaN,NaN,NaN
6,N01-005,Sample_Fq26,Blood,Control,GPA control,2,yes,NaN,old,COMBIVAS,NaN,NaN,NaN,NaN
7,N01-011,Sample_Fq60,Blood,Control,GPA control,4,yes,NaN,middle,COMBIVAS,NaN,NaN,NaN,NaN
8,1,Sample_Fq1_COV,Blood,COVID,Active COV,c1,severe,severe,Old,COVID,M,NaN,NaN,NaN
9,1,Sample_Fq2_COV,Nasal,COVID,Active COV,c1,severe,severe,Old,COVID,M,NaN,NaN,NaN


In [6]:
sampleList.sampleid

0        Sample_Fq21
1         Sample_Fq2
2        Sample_Fq27
3        Sample_Fq61
4        Sample_Fq20
5         Sample_Fq1
6        Sample_Fq26
7        Sample_Fq60
8     Sample_Fq1_COV
9     Sample_Fq2_COV
10    Sample_Fq3_COV
11    Sample_Fq4_COV
12    Sample_Fq5_COV
13    Sample_Fq6_COV
14       Sample_Fq_4
15       Sample_Fq_5
16      Sample_Fq_10
17      Sample_Fq_11
18      Sample_Fq_14
19      Sample_Fq_15
20      Sample_Fq_18
21      Sample_Fq_19
22      Sample_Fq_38
23         MC9785359
24         MC9785360
25         MC9785361
26         MC9785362
27         MC9785363
28         MC9785364
29         MC9785365
30         MC9785366
31         MC9785367
32         MC9785368
33         MC9785369
34         MC9785370
35         MC9785371
36         MC9785372
37         MC9785373
38         MC9785374
39         MC9785407
Name: sampleid, dtype: object

In [7]:
#Create a list of samples
samplelist = sampleList['sampleid'].tolist()
samplelist

['Sample_Fq21',
 'Sample_Fq2',
 'Sample_Fq27',
 'Sample_Fq61',
 'Sample_Fq20',
 'Sample_Fq1',
 'Sample_Fq26',
 'Sample_Fq60',
 'Sample_Fq1_COV',
 'Sample_Fq2_COV',
 'Sample_Fq3_COV',
 'Sample_Fq4_COV',
 'Sample_Fq5_COV',
 'Sample_Fq6_COV',
 'Sample_Fq_4',
 'Sample_Fq_5',
 'Sample_Fq_10',
 'Sample_Fq_11',
 'Sample_Fq_14',
 'Sample_Fq_15',
 'Sample_Fq_18',
 'Sample_Fq_19',
 'Sample_Fq_38',
 'MC9785359',
 'MC9785360',
 'MC9785361',
 'MC9785362',
 'MC9785363',
 'MC9785364',
 'MC9785365',
 'MC9785366',
 'MC9785367',
 'MC9785368',
 'MC9785369',
 'MC9785370',
 'MC9785371',
 'MC9785372',
 'MC9785373',
 'MC9785374',
 'MC9785407']

In [8]:
#Import data in samplelist using dandelion standard qc package
for x in sampleList.sampleid:
    adata_tmp[x] = sc.read_10x_mtx("COVID_combined_Trans/" +x+ '/filtered_feature_bc_matrix/')
    ddl.pp.external.recipe_scanpy_qc(adata_tmp[x])
adata_tmp

/Users/matthewcoates/opt/anaconda3/envs/dandelion/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:810: UserWarning: Revieved a view of an AnnData. Making a copy.
/Users/matthewcoates/opt/anaconda3/envs/dandelion/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:810: UserWarning: Revieved a view of an AnnData. Making a copy.
/Users/matthewcoates/opt/anaconda3/envs/dandelion/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:810: UserWarning: Revieved a view of an AnnData. Making a copy.
/Users/matthewcoates/opt/anaconda3/envs/dandelion/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:810: UserWarning: Revieved a view of an AnnData. Making a copy.
/Users/matthewcoates/opt/anaconda3/envs/dandelion/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:810: UserWarning: Revieved a view of an AnnData. Making a copy.
/Users/matthewcoates/opt/anaconda3/envs/dandelion/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:810: UserWarning: Re

defaultdict(dict,
            {'Sample_Fq21': AnnData object with n_obs × n_vars = 5452 × 33538
                 obs: 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna'
                 var: 'gene_ids', 'feature_types',
             'Sample_Fq2': AnnData object with n_obs × n_vars = 10798 × 33538
                 obs: 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna'
                 var: 'gene_ids', 'feature_types',
             'Sample_Fq27': AnnData object with n_obs × n_vars = 1569 × 33538
                 obs: 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna'
                 var: 'gene_ids', 'feature_types',
             'Sample_Fq61': AnnData object with n_obs × n_vars = 4660 × 33538
                 obs: 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna'
                 var: 'gene_ids', 'feature_types',
             'Sample_Fq20': AnnData object

In [9]:
#View dandelion import of objects in samplelist
adata_tmp

defaultdict(dict,
            {'Sample_Fq21': AnnData object with n_obs × n_vars = 5452 × 33538
                 obs: 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna'
                 var: 'gene_ids', 'feature_types',
             'Sample_Fq2': AnnData object with n_obs × n_vars = 10798 × 33538
                 obs: 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna'
                 var: 'gene_ids', 'feature_types',
             'Sample_Fq27': AnnData object with n_obs × n_vars = 1569 × 33538
                 obs: 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna'
                 var: 'gene_ids', 'feature_types',
             'Sample_Fq61': AnnData object with n_obs × n_vars = 4660 × 33538
                 obs: 'scrublet_score', 'n_genes', 'percent_mito', 'n_counts', 'is_doublet', 'filter_rna'
                 var: 'gene_ids', 'feature_types',
             'Sample_Fq20': AnnData object

In [10]:
"""Functions for downstream work with outputs of remove-background for cellbender data."""

import tables
import numpy as np
import scipy.sparse as sp
from typing import Dict


def dict_from_h5(file: str) -> Dict[str, np.ndarray]:
    """Read in everything from an h5 file and put into a dictionary."""
    d = {}
    with tables.open_file(file) as f:
        # read in everything
        for array in f.walk_nodes("/", "Array"):
            d[array.name] = array.read()
    return d


def anndata_from_h5(file: str,
                    analyzed_barcodes_only: bool = True) -> 'anndata.AnnData':
    """Load an output h5 file into an AnnData object for downstream work.

    Args:
        file: The h5 file
        analyzed_barcodes_only: False to load all barcodes, so that the size of
            the AnnData object will match the size of the input raw count matrix.
            True to load a limited set of barcodes: only those analyzed by the
            algorithm. This allows relevant latent variables to be loaded
            properly into adata.obs and adata.obsm, rather than adata.uns.

    Returns:
        adata: The anndata object, populated with inferred latent variables
            and metadata.

    """

    try:
        import anndata
    except ImportError:
        raise ImportError('The anndata package must be installed to use the '
                          'function anndata_from_h5()')

    d = dict_from_h5(file)
    X = sp.csc_matrix((d.pop('data'), d.pop('indices'), d.pop('indptr')),
                      shape=d.pop('shape')).transpose().tocsr()

    if analyzed_barcodes_only:
        if 'barcodes_analyzed_inds' in d.keys():
            X = X[d['barcodes_analyzed_inds'], :]
            d['barcodes'] = d['barcodes'][d['barcodes_analyzed_inds']]
        elif 'barcode_indices_for_latents' in d.keys():
            X = X[d['barcode_indices_for_latents'], :]
            d['barcodes'] = d['barcodes'][d['barcode_indices_for_latents']]
        else:
            print('Warning: analyzed_barcodes_only=True, but the key '
                  '"barcodes_analyzed_inds" or "barcode_indices_for_latents" '
                  'is missing from the h5 file. '
                  'Will output all barcodes, and proceed as if '
                  'analyzed_barcodes_only=False')

    # Construct the count matrix.
    adata = anndata.AnnData(X=X,
                            obs={'barcode': d.pop('barcodes').astype(str)},
                            var={'gene_name': (d.pop('gene_names') if 'gene_names' in d.keys()
                                               else d.pop('name')).astype(str)})
    adata.obs.set_index('barcode', inplace=True)
    adata.var.set_index('gene_name', inplace=True)

    # Add other information to the adata object in the appropriate slot.
    for key, value in d.items():
        try:
            value = np.asarray(value)
            if len(value.shape) == 0:
                adata.uns[key] = value
            elif value.shape[0] == X.shape[0]:
                if (len(value.shape) < 2) or (value.shape[1] < 2):
                    adata.obs[key] = value
                else:
                    adata.obsm[key] = value
            elif value.shape[0] == X.shape[1]:
                if value.dtype.name.startswith('bytes'):
                    adata.var[key] = value.astype(str)
                else:
                    adata.var[key] = value
            else:
                adata.uns[key] = value
        except Exception:
            print('Unable to load data into AnnData: ', key, value, type(value))

    if analyzed_barcodes_only:
        for col in adata.obs.columns[adata.obs.columns.str.startswith('barcodes_analyzed')
                                     | adata.obs.columns.str.startswith('barcode_indices')]:
            try:
                del adata.obs[col]
            except Exception:
                pass

    return adata


def load_anndata_from_input_and_output(input_file: str,
                                       output_file: str,
                                       analyzed_barcodes_only: bool = True,
                                       input_layer_key: str = 'cellranger') -> 'anndata.AnnData':
    """Load remove-background output count matrix into an anndata object,
    together with remove-background metadata and the raw input counts.

    Args:
        input_file: Raw h5 file used as input for remove-background.
        output_file: Output h5 file created by remove-background (can be
            filtered or not).
        analyzed_barcodes_only: Argument passed to anndata_from_h5().
            False to load all barcodes, so that the size of
            the AnnData object will match the size of the input raw count matrix.
            True to load a limited set of barcodes: only those analyzed by the
            algorithm. This allows relevant latent variables to be loaded
            properly into adata.obs and adata.obsm, rather than adata.uns.
        input_layer_key: Key of the anndata.layer that is created for the raw
            input count matrix.

    Return:
        adata_out: AnnData object with counts before and after remove-background,
            as well as inferred latent variables from remove-background.

    """

    # Load input data.
    adata_raw = anndata_from_h5(input_file, analyzed_barcodes_only=False)

    # Load remove-background output data.
    adata_out = anndata_from_h5(output_file, analyzed_barcodes_only=analyzed_barcodes_only)

    # Subset the raw dataset to the relevant barcodes.
    adata_raw = adata_raw[adata_out.obs.index]

    # Put count matrices into 'layers' in anndata for clarity.
    adata_out.layers[input_layer_key] = adata_raw.X.copy()
    adata_out.layers['cellbender'] = adata_out.X.copy()

    # Pre-compute a bit of metadata.
    adata_out.var['n_cellranger'] = np.array(adata_out.layers['cellranger'].sum(axis=0)).squeeze()
    adata_out.var['n_cellbender'] = np.array(adata_out.layers['cellbender'].sum(axis=0)).squeeze()

    return adata_out



In [11]:
#View samplelist for list of samples to be imported
samplelist

['Sample_Fq21',
 'Sample_Fq2',
 'Sample_Fq27',
 'Sample_Fq61',
 'Sample_Fq20',
 'Sample_Fq1',
 'Sample_Fq26',
 'Sample_Fq60',
 'Sample_Fq1_COV',
 'Sample_Fq2_COV',
 'Sample_Fq3_COV',
 'Sample_Fq4_COV',
 'Sample_Fq5_COV',
 'Sample_Fq6_COV',
 'Sample_Fq_4',
 'Sample_Fq_5',
 'Sample_Fq_10',
 'Sample_Fq_11',
 'Sample_Fq_14',
 'Sample_Fq_15',
 'Sample_Fq_18',
 'Sample_Fq_19',
 'Sample_Fq_38',
 'MC9785359',
 'MC9785360',
 'MC9785361',
 'MC9785362',
 'MC9785363',
 'MC9785364',
 'MC9785365',
 'MC9785366',
 'MC9785367',
 'MC9785368',
 'MC9785369',
 'MC9785370',
 'MC9785371',
 'MC9785372',
 'MC9785373',
 'MC9785374',
 'MC9785407']

In [12]:
#Create blank dictionary for cellbender data
adata_cellbender = defaultdict(dict)

In [13]:
#Testing Ben's function on one data path
adata_cellbender = load_anndata_from_input_and_output(input_file = ("COVID_combined_trans/Sample_Fq21/raw_feature_bc_matrix.h5"),
                                       output_file = ("COVID_combined_trans/Sample_Fq21/cellbender_out/output.h5"))
adata_cellbender

AnnData object with n_obs × n_vars = 55034 × 33538
    obs: 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale'
    var: 'ambient_expression', 'feature_type', 'genome', 'id', 'n_cellranger', 'n_cellbender'
    uns: 'contamination_fraction_params', 'fraction_data_used_for_testing', 'lambda_multiplier', 'overdispersion_mean_and_scale', 'target_false_positive_rate', 'test_elbo', 'test_epoch', 'training_elbo_per_epoch'
    obsm: 'latent_gene_encoding'
    layers: 'cellranger', 'cellbender'

In [38]:
#Create blank dictionaries for filepath lists
input_cfile = defaultdict(dict)
output_cfile = defaultdict(dict)
input_newfile = defaultdict(dict)

In [15]:
input_cfile

defaultdict(dict, {})

In [39]:
#Create filepaths for list
for x in sampleList.sampleid:
    input_cfile[x] = ("COVID_combined_trans/" +x+ "/raw_feature_bc_matrix.h5")
    output_cfile[x] = ("COVID_combined_trans/" +x+ "/cellbender_out/output.h5")
    input_newfile[x] = ("COVID_combined_trans/" +x+ "/cellbender_out/output_filtered.h5")
input_newfile["Sample_Fq21"]

'COVID_combined_trans/Sample_Fq21/cellbender_out/output_filtered.h5'

In [17]:
#Re-Create blank dictionary for cellbender object
adata_cellbender = defaultdict(dict)

In [19]:
#Run Ben's 'load_anndata_from_input_and_output' cellbender function to collate cellbender data into anndata object
for x in samplelist:
    adata_cellbender[x] = load_anndata_from_input_and_output(input_file = input_cfile[x], output_file = output_cfile[x])
adata_cellbender

defaultdict(dict,
            {'Sample_Fq21': AnnData object with n_obs × n_vars = 55034 × 33538
                 obs: 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale'
                 var: 'ambient_expression', 'feature_type', 'genome', 'id', 'n_cellranger', 'n_cellbender'
                 uns: 'contamination_fraction_params', 'fraction_data_used_for_testing', 'lambda_multiplier', 'overdispersion_mean_and_scale', 'target_false_positive_rate', 'test_elbo', 'test_epoch', 'training_elbo_per_epoch'
                 obsm: 'latent_gene_encoding'
                 layers: 'cellranger', 'cellbender',
             'Sample_Fq2': AnnData object with n_obs × n_vars = 75155 × 33547
                 obs: 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale'
                 var: 'ambient_expression', 'feature_type', 'genome', 'id', 'n_cellranger', 'n_cellbender'
                 uns: 'contamination_fraction_params', 'fraction_data_used_for_testing', 'lambda_multiplier', 

In [28]:
#Example of one sample
adata_cellbender['Sample_Fq21']

AnnData object with n_obs × n_vars = 55034 × 33538
    obs: 'latent_RT_efficiency', 'latent_cell_probability', 'latent_scale'
    var: 'ambient_expression', 'feature_type', 'genome', 'id', 'n_cellranger', 'n_cellbender'
    uns: 'contamination_fraction_params', 'fraction_data_used_for_testing', 'lambda_multiplier', 'overdispersion_mean_and_scale', 'target_false_positive_rate', 'test_elbo', 'test_epoch', 'training_elbo_per_epoch'
    obsm: 'latent_gene_encoding'
    layers: 'cellranger', 'cellbender'